# Init Spark

In [136]:
import pyspark
from pyspark.sql import SparkSession

In [137]:
spark = ( 
    SparkSession.builder
            .appName('test').master("yarn")
            .getOrCreate()
)

.master("local[*]").  Nếu chạy local. Dấu * tượng trưng cho ý spark được phép dùng hết tài nguyên của máy ( CPU/RAM ) để xử lý 

.master("yarn"). Nếu chạy trên cluster YARN


Có thể thêm config vào. Một số config để connect với s3

  - .config("spark.jars", "aws-sdk-java-2.17.81.jar") 

  - .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
  
  - .config("spark.hadoop.fs.s3a.access.key", "<your_access_key_id>")
  
  - .config("spark.hadoop.fs.s3a.secret.key", "<your_secret_access_key>")

In [138]:
spark

In [139]:
!pyspark --version and spark-shell --version 

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.0
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.20.1
Branch dataproc-branch-3.5.0
Compiled by user  on 2024-03-06T22:47:14Z
Revision 8c74211db15b74df03f0cfb2fd46a60ebe95f5c8
Url https://bigdataoss-internal.googlesource.com/third_party/apache/spark
Type --help for more information.


# Read File

In [140]:
data_storage = "gs://thuvienphapluat_crawl/2024/03/22/HuynhHuuQuoc_First.parquet"

df = spark.read.parquet(data_storage)

In [141]:
df.printSchema()

root
 |-- Link: string (nullable = true)
 |-- Lawname: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Answer: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [142]:
df.show()

+--------------------+--------+--------------------+--------------------+--------------------+
|                Link| Lawname|               Title|            Question|              Answer|
+--------------------+--------+--------------------+--------------------+--------------------+
|https://thuvienph...|phapluat|Bằng độc quyền gi...|Bằng độc quyền gi...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Quyền công bố tác...|Tôi có một câu hỏ...|[Quyền nhân thân,...|
|https://thuvienph...|phapluat|Thời hạn bảo hộ q...|Tôi có một câu hỏ...|[Các loại hình tá...|
|https://thuvienph...|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Công nghệ mới thu...|Tôi có thắc mắc: ...|[Công nghệ khuyến...|
|https://thuvienph...|phapluat|Chủ sở hữu bí mật...|Chủ sở hữu bí mật...|[Chủ sở hữu đối t...|
|https://thuvienph...|phapluat|Bí mật về nhân th...|Bí mật về nhân th...|[Đối tượng không ...|
|https://thuvienph...|phapluat|Chỉ dẫn địa lý đồ..

# Read Spark with Schema

In [143]:
from pyspark.sql import types

In [144]:
schema = types.StructType([
    types.StructField('Link', types.StringType(), True),
    types.StructField('Lawname', types.StringType(), True),
    types.StructField('Title', types.StringType(), True),
    types.StructField('Question', types.StringType(), True),
    types.StructField('Answer', types.ArrayType(types.StringType()), True),  # Sử dụng ArrayType cho danh sách các chuỗi
])

In [145]:
df = spark.read \
    .schema(schema) \
    .parquet(data_storage)

In [146]:
df.show()

+--------------------+--------+--------------------+--------------------+--------------------+
|                Link| Lawname|               Title|            Question|              Answer|
+--------------------+--------+--------------------+--------------------+--------------------+
|https://thuvienph...|phapluat|Bằng độc quyền gi...|Bằng độc quyền gi...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Quyền công bố tác...|Tôi có một câu hỏ...|[Quyền nhân thân,...|
|https://thuvienph...|phapluat|Thời hạn bảo hộ q...|Tôi có một câu hỏ...|[Các loại hình tá...|
|https://thuvienph...|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Công nghệ mới thu...|Tôi có thắc mắc: ...|[Công nghệ khuyến...|
|https://thuvienph...|phapluat|Chủ sở hữu bí mật...|Chủ sở hữu bí mật...|[Chủ sở hữu đối t...|
|https://thuvienph...|phapluat|Bí mật về nhân th...|Bí mật về nhân th...|[Đối tượng không ...|
|https://thuvienph...|phapluat|Chỉ dẫn địa lý đồ..

# Làm việc với cột ( columns ) 

In [147]:
df = df.withColumnRenamed("Link", "Link_TVPL")

In [148]:
df.show()

+--------------------+--------+--------------------+--------------------+--------------------+
|           Link_TVPL| Lawname|               Title|            Question|              Answer|
+--------------------+--------+--------------------+--------------------+--------------------+
|https://thuvienph...|phapluat|Bằng độc quyền gi...|Bằng độc quyền gi...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Quyền công bố tác...|Tôi có một câu hỏ...|[Quyền nhân thân,...|
|https://thuvienph...|phapluat|Thời hạn bảo hộ q...|Tôi có một câu hỏ...|[Các loại hình tá...|
|https://thuvienph...|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Công nghệ mới thu...|Tôi có thắc mắc: ...|[Công nghệ khuyến...|
|https://thuvienph...|phapluat|Chủ sở hữu bí mật...|Chủ sở hữu bí mật...|[Chủ sở hữu đối t...|
|https://thuvienph...|phapluat|Bí mật về nhân th...|Bí mật về nhân th...|[Đối tượng không ...|
|https://thuvienph...|phapluat|Chỉ dẫn địa lý đồ..

In [149]:
df.select("Lawname","Title").describe().show()

+-------+--------+--------------------+
|summary| Lawname|               Title|
+-------+--------+--------------------+
|  count|      19|                  19|
|   mean|    NULL|                NULL|
| stddev|    NULL|                NULL|
|    min|phapluat|Biên bản cung cấp...|
|    max|phapluat|Tài liệu chứng mi...|
+-------+--------+--------------------+



In [150]:
df.printSchema()

root
 |-- Link_TVPL: string (nullable = true)
 |-- Lawname: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Answer: array (nullable = true)
 |    |-- element: string (containsNull = true)



# Pyspark và SQL

### SELECT

In [151]:
df.select("*").show()

+--------------------+--------+--------------------+--------------------+--------------------+
|           Link_TVPL| Lawname|               Title|            Question|              Answer|
+--------------------+--------+--------------------+--------------------+--------------------+
|https://thuvienph...|phapluat|Bằng độc quyền gi...|Bằng độc quyền gi...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Quyền công bố tác...|Tôi có một câu hỏ...|[Quyền nhân thân,...|
|https://thuvienph...|phapluat|Thời hạn bảo hộ q...|Tôi có một câu hỏ...|[Các loại hình tá...|
|https://thuvienph...|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Công nghệ mới thu...|Tôi có thắc mắc: ...|[Công nghệ khuyến...|
|https://thuvienph...|phapluat|Chủ sở hữu bí mật...|Chủ sở hữu bí mật...|[Chủ sở hữu đối t...|
|https://thuvienph...|phapluat|Bí mật về nhân th...|Bí mật về nhân th...|[Đối tượng không ...|
|https://thuvienph...|phapluat|Chỉ dẫn địa lý đồ..

In [152]:
df.select("*").head()

Row(Link_TVPL='https://thuvienphapluat.vn/phap-luat/bang-doc-quyen-giai-phap-huu-ich-co-hieu-luc-bao-lau-va-bat-dau-co-hieu-luc-tu-thoi-diem-nao-theo-q-864416-141573.html', Lawname='phapluat', Title='Bằng độc quyền giải pháp hữu ích có hiệu lực bao lâu và bắt đầu có hiệu lực từ thời điểm nào theo quy định?', Question='Bằng độc quyền giải pháp hữu ích là văn bằng bảo hộ đúng không? Bằng độc quyền giải pháp hữu ích có hiệu lực bao lâu và bắt đầu có hiệu lực từ thời điểm nào theo quy định? câu hỏi của anh N (Nha Trang).', Answer=['Văn bằng bảo hộ', '1. Văn bằng bảo hộ ghi nhận chủ sở hữu sáng chế, kiểu dáng công nghiệp, thiết kế bố trí, nhãn hiệu (sau đây gọi là chủ văn bằng bảo hộ); tác giả sáng chế, kiểu dáng công nghiệp, thiết kế bố trí; đối tượng, phạm vi và thời hạn bảo hộ.', '2. Văn bằng bảo hộ chỉ dẫn địa lý ghi nhận tổ chức quản lý chỉ dẫn địa lý, chỉ dẫn địa lý được bảo hộ, tính chất đặc thù của sản phẩm mang chỉ dẫn địa lý, tính chất đặc thù về điều kiện địa lý và khu vực địa lý

In [153]:
df.select(
    "Lawname",
    "Title",
    "Question",
    "Answer"
    ).show()

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Bằng độc quyền gi...|Bằng độc quyền gi...|[Văn bằng bảo hộ,...|
|phapluat|Quyền công bố tác...|Tôi có một câu hỏ...|[Quyền nhân thân,...|
|phapluat|Thời hạn bảo hộ q...|Tôi có một câu hỏ...|[Các loại hình tá...|
|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
|phapluat|Công nghệ mới thu...|Tôi có thắc mắc: ...|[Công nghệ khuyến...|
|phapluat|Chủ sở hữu bí mật...|Chủ sở hữu bí mật...|[Chủ sở hữu đối t...|
|phapluat|Bí mật về nhân th...|Bí mật về nhân th...|[Đối tượng không ...|
|phapluat|Chỉ dẫn địa lý đồ...|Chỉ dẫn địa lý đồ...|[Điều kiện chung ...|
|phapluat|Biên bản cung cấp...|Cho tôi hỏi việc ...|[Chuẩn bị thực hi...|
|phapluat|Trong các hình th...|Tôi có thắc mắc n...|[Giải thích từ ng...|
|phapluat|Ranh giới của khu...|Ranh gi

In [154]:
df.select("Title").distinct().show()

+--------------------+
|               Title|
+--------------------+
|Biên bản cung cấp...|
|Chủ sở hữu bí mật...|
|Công nghệ mới thu...|
|Giấy chứng nhận đ...|
|Người yêu cầu áp ...|
|Tài liệu chứng mi...|
|Chỉ dẫn địa lý đồ...|
|Thời hạn bảo hộ q...|
|Bí mật về nhân th...|
|Quyền công bố tác...|
|Trong các hình th...|
|Phải nộp phí duy ...|
|Bằng độc quyền gi...|
|Thời hạn của bằng...|
|Nhãn hiệu chứng n...|
|Mẫu quyết định th...|
|Mẫu Tờ khai đăng ...|
|Chủ sở hữu sáng c...|
|Ranh giới của khu...|
+--------------------+



In [155]:
from pyspark.sql import functions as F

In [156]:
df.select(
    F.col("Lawname"),
    F.col("Title"),
).distinct().show()

+--------+--------------------+
| Lawname|               Title|
+--------+--------------------+
|phapluat|Nhãn hiệu chứng n...|
|phapluat|Thời hạn bảo hộ q...|
|phapluat|Trong các hình th...|
|phapluat|Chỉ dẫn địa lý đồ...|
|phapluat|Chủ sở hữu bí mật...|
|phapluat|Mẫu quyết định th...|
|phapluat|Tài liệu chứng mi...|
|phapluat|Mẫu Tờ khai đăng ...|
|phapluat|Bí mật về nhân th...|
|phapluat|Người yêu cầu áp ...|
|phapluat|Giấy chứng nhận đ...|
|phapluat|Thời hạn của bằng...|
|phapluat|Bằng độc quyền gi...|
|phapluat|Quyền công bố tác...|
|phapluat|Biên bản cung cấp...|
|phapluat|Phải nộp phí duy ...|
|phapluat|Công nghệ mới thu...|
|phapluat|Chủ sở hữu sáng c...|
|phapluat|Ranh giới của khu...|
+--------+--------------------+



### WHERE

In [157]:
df.select(
    F.col("Lawname"),
    F.col("Title"),
    F.col("Question"),
    F.col("Answer")
).filter(F.col("Question").contains("Giấy chứng nhận đăng ký nhãn hiệu có hiệu lực từ thời điểm nào? Hiệu lực của loại GCN này sau khi gia hạn được quy định ra sao? Mỗi đơn đăng ký sở hữu công nghiệp có thể yêu cầu cấp Giấy chứng nhận đăng ký nhãn hiệu cho mấy nhãn hiệu? Câu hỏi của chị L (Đồng Nai).")).show()

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
+--------+--------------------+--------------------+--------------------+



In [158]:
condition = (F.col("Lawname").contains("phapluat")) & (F.col("Question").contains("Giấy chứng nhận đăng ký nhãn hiệu có hiệu lực từ thời điểm nào? Hiệu lực của loại GCN này sau khi gia hạn được quy định ra sao? Mỗi đơn đăng ký sở hữu công nghiệp có thể yêu cầu cấp Giấy chứng nhận đăng ký nhãn hiệu cho mấy nhãn hiệu? Câu hỏi của chị L (Đồng Nai)."))

df.select(
    F.col("Lawname"),
    F.col("Title"),
    F.col("Question"),
    F.col("Answer")
).filter(condition).show()

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
+--------+--------------------+--------------------+--------------------+



### ORDER BY

In [159]:
df.select(
    F.col("Lawname"),
    F.col("Title"),
    F.col("Question"),
    F.col("Answer")
).filter(F.col("Lawname").contains("phapluat")).orderBy("Title").show()

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Biên bản cung cấp...|Cho tôi hỏi việc ...|[Chuẩn bị thực hi...|
|phapluat|Bí mật về nhân th...|Bí mật về nhân th...|[Đối tượng không ...|
|phapluat|Bằng độc quyền gi...|Bằng độc quyền gi...|[Văn bằng bảo hộ,...|
|phapluat|Chỉ dẫn địa lý đồ...|Chỉ dẫn địa lý đồ...|[Điều kiện chung ...|
|phapluat|Chủ sở hữu bí mật...|Chủ sở hữu bí mật...|[Chủ sở hữu đối t...|
|phapluat|Chủ sở hữu sáng c...|Tôi có thắc mắc m...|[Các yếu tố hạn c...|
|phapluat|Công nghệ mới thu...|Tôi có thắc mắc: ...|[Công nghệ khuyến...|
|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
|phapluat|Mẫu Tờ khai đăng ...|Mẫu Tờ khai đăng ...|[Các mẫu ban hành...|
|phapluat|Mẫu quyết định th...|Cho tôi hỏi mẫu q...|[Chuẩn bị thực hi...|
|phapluat|Người yêu cầu áp ...|Cho tôi

In [160]:
df.select(
    F.col("Lawname"),
    F.col("Title"),
    F.col("Question"),
    F.col("Answer")
).filter(F.col("Lawname").contains("phapluat")).sort("Title").show()

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Biên bản cung cấp...|Cho tôi hỏi việc ...|[Chuẩn bị thực hi...|
|phapluat|Bí mật về nhân th...|Bí mật về nhân th...|[Đối tượng không ...|
|phapluat|Bằng độc quyền gi...|Bằng độc quyền gi...|[Văn bằng bảo hộ,...|
|phapluat|Chỉ dẫn địa lý đồ...|Chỉ dẫn địa lý đồ...|[Điều kiện chung ...|
|phapluat|Chủ sở hữu bí mật...|Chủ sở hữu bí mật...|[Chủ sở hữu đối t...|
|phapluat|Chủ sở hữu sáng c...|Tôi có thắc mắc m...|[Các yếu tố hạn c...|
|phapluat|Công nghệ mới thu...|Tôi có thắc mắc: ...|[Công nghệ khuyến...|
|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
|phapluat|Mẫu Tờ khai đăng ...|Mẫu Tờ khai đăng ...|[Các mẫu ban hành...|
|phapluat|Mẫu quyết định th...|Cho tôi hỏi mẫu q...|[Chuẩn bị thực hi...|
|phapluat|Người yêu cầu áp ...|Cho tôi

In [161]:
df.select(
    F.col("Lawname"),
    F.col("Title"),
    F.col("Question"),
    F.col("Answer")
).filter(F.col("Lawname").contains("phapluat")).sort(F.col("Title").desc()).show()

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Tài liệu chứng mi...|Tôi có thắc mắc l...|[Căn cứ, thủ tục ...|
|phapluat|Trong các hình th...|Tôi có thắc mắc n...|[Giải thích từ ng...|
|phapluat|Thời hạn của bằng...|Thời hạn của bằng...|[Sửa đổi, cấp lại...|
|phapluat|Thời hạn bảo hộ q...|Tôi có một câu hỏ...|[Các loại hình tá...|
|phapluat|Ranh giới của khu...|Ranh giới của khu...|[Khu vực địa lý m...|
|phapluat|Quyền công bố tác...|Tôi có một câu hỏ...|[Quyền nhân thân,...|
|phapluat|Phải nộp phí duy ...|Cho tôi hỏi trong...|[Văn bằng bảo hộ,...|
|phapluat|Nhãn hiệu chứng n...|Tôi có một số thắ...|[18. Nhãn hiệu ch...|
|phapluat|Người yêu cầu áp ...|Cho tôi hỏi: Ngườ...|[Nghĩa vụ của ngư...|
|phapluat|Mẫu quyết định th...|Cho tôi hỏi mẫu q...|[Chuẩn bị thực hi...|
|phapluat|Mẫu Tờ khai đăng ...|Mẫu Tờ 

In [162]:
(
   df.select(
    F.col("Lawname"),
    F.col("Title"),
    F.col("Question"),
    F.col("Answer")
    )
    .filter(F.col("Lawname").contains("phapluat"))
    .sort(F.desc("Title"))
    .show()
)

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Tài liệu chứng mi...|Tôi có thắc mắc l...|[Căn cứ, thủ tục ...|
|phapluat|Trong các hình th...|Tôi có thắc mắc n...|[Giải thích từ ng...|
|phapluat|Thời hạn của bằng...|Thời hạn của bằng...|[Sửa đổi, cấp lại...|
|phapluat|Thời hạn bảo hộ q...|Tôi có một câu hỏ...|[Các loại hình tá...|
|phapluat|Ranh giới của khu...|Ranh giới của khu...|[Khu vực địa lý m...|
|phapluat|Quyền công bố tác...|Tôi có một câu hỏ...|[Quyền nhân thân,...|
|phapluat|Phải nộp phí duy ...|Cho tôi hỏi trong...|[Văn bằng bảo hộ,...|
|phapluat|Nhãn hiệu chứng n...|Tôi có một số thắ...|[18. Nhãn hiệu ch...|
|phapluat|Người yêu cầu áp ...|Cho tôi hỏi: Ngườ...|[Nghĩa vụ của ngư...|
|phapluat|Mẫu quyết định th...|Cho tôi hỏi mẫu q...|[Chuẩn bị thực hi...|
|phapluat|Mẫu Tờ khai đăng ...|Mẫu Tờ 

###  Limit 

In [163]:
(
    df.select(
    F.col("Lawname"),
    F.col("Title"),
    F.col("Question"),
    F.col("Answer")
    )
    .filter(F.col("Lawname").contains("phapluat"))
    .sort(F.col("Title"))
    .limit(5)
    .show()
)

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Biên bản cung cấp...|Cho tôi hỏi việc ...|[Chuẩn bị thực hi...|
|phapluat|Bí mật về nhân th...|Bí mật về nhân th...|[Đối tượng không ...|
|phapluat|Bằng độc quyền gi...|Bằng độc quyền gi...|[Văn bằng bảo hộ,...|
|phapluat|Chỉ dẫn địa lý đồ...|Chỉ dẫn địa lý đồ...|[Điều kiện chung ...|
|phapluat|Chủ sở hữu bí mật...|Chủ sở hữu bí mật...|[Chủ sở hữu đối t...|
+--------+--------------------+--------------------+--------------------+



### GROUP BY 

In [164]:
 (
    df.select(
        F.col("Lawname"),
        F.col("Title")
    )
     .groupBy("Lawname")
     .count()
     .show()
 )

+--------+-----+
| Lawname|count|
+--------+-----+
|phapluat|   19|
+--------+-----+



# User Define Function ( UDF )

In [165]:
def convert_case(string):
    return string.upper()

convert_case_udf = F.udf(convert_case, returnType=types.StringType())

In [166]:
( 
    df
    .withColumn('instructors_uppercase', convert_case_udf(F.col("Lawname")))
    .select('instructors_uppercase','Lawname') 
    .show()
)

+---------------------+--------+
|instructors_uppercase| Lawname|
+---------------------+--------+
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
|             PHAPLUAT|phapluat|
+---------------------+--------+



# Spark SQL

In [167]:
df.createOrReplaceTempView("df")
# df.registerTempTable("df")

In [168]:
spark.sql(" SELECT * from df ").show()

+--------------------+--------+--------------------+--------------------+--------------------+
|           Link_TVPL| Lawname|               Title|            Question|              Answer|
+--------------------+--------+--------------------+--------------------+--------------------+
|https://thuvienph...|phapluat|Bằng độc quyền gi...|Bằng độc quyền gi...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Quyền công bố tác...|Tôi có một câu hỏ...|[Quyền nhân thân,...|
|https://thuvienph...|phapluat|Thời hạn bảo hộ q...|Tôi có một câu hỏ...|[Các loại hình tá...|
|https://thuvienph...|phapluat|Giấy chứng nhận đ...|Giấy chứng nhận đ...|[Văn bằng bảo hộ,...|
|https://thuvienph...|phapluat|Công nghệ mới thu...|Tôi có thắc mắc: ...|[Công nghệ khuyến...|
|https://thuvienph...|phapluat|Chủ sở hữu bí mật...|Chủ sở hữu bí mật...|[Chủ sở hữu đối t...|
|https://thuvienph...|phapluat|Bí mật về nhân th...|Bí mật về nhân th...|[Đối tượng không ...|
|https://thuvienph...|phapluat|Chỉ dẫn địa lý đồ..

In [169]:
query = """
SELECT Lawname, Title, Question, Answer
FROM df WHERE Question == 'Tôi có thắc mắc muốn được giải đáp như sau chủ sở hữu sáng chế, kiểu dáng công nghiệp, thiết kế bố trí phải trả thù lao cho các đồng tác giả khi không có thỏa thuận như thế nào? Câu hỏi của anh A.L.Q đến từ TP.HCM.'

""" 
spark.sql(query).show()


+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Chủ sở hữu sáng c...|Tôi có thắc mắc m...|[Các yếu tố hạn c...|
+--------+--------------------+--------------------+--------------------+



In [170]:
query = """

SELECT Lawname, Title, Question,Answer
FROM df WHERE Question == 'Tôi có thắc mắc muốn được giải đáp như sau chủ sở hữu sáng chế, kiểu dáng công nghiệp, thiết kế bố trí phải trả thù lao cho các đồng tác giả khi không có thỏa thuận như thế nào? Câu hỏi của anh A.L.Q đến từ TP.HCM.'

""" 

spark.sql(query).write.parquet("gs://thuvienphapluat_crawl/2024/03/22/HuynhHuuQuoc_LAST.parquet",
                               mode = "overwrite")

# Dùng  write.partitionBy(col).parquet để partition

# Ghi kết quả ra file parquet 

In [171]:
result_df = spark.read.parquet("gs://thuvienphapluat_crawl/2024/03/22/HuynhHuuQuoc_LAST.parquet")

In [172]:
result_df.show()

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Chủ sở hữu sáng c...|Tôi có thắc mắc m...|[Các yếu tố hạn c...|
+--------+--------------------+--------------------+--------------------+



# Ví dụ thực tế.

In [173]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types

spark = ( 
    SparkSession.builder
            .master("yarn") 
            .appName('Thu Vien Phap Luat')
            .getOrCreate()
)

schema = types.StructType([
    types.StructField('Link', types.StringType(), True),
    types.StructField('Lawname', types.StringType(), True),
    types.StructField('Title', types.StringType(), True),
    types.StructField('Question', types.StringType(), True),
    types.StructField('Answer', types.StringType(), True)

])

data_storage = "gs://thuvienphapluat_crawl/2024/03/22/HuynhHuuQuoc_First.parquet"
data_write = "gs://thuvienphapluat_crawl/2024/03/22/thu_vien_phap_luat"

df = spark.read.parquet(data_storage)

df.createOrReplaceTempView("df")
query = """

SELECT Lawname, Title, Question, Answer
FROM df WHERE Question == 'Tôi có thắc mắc muốn được giải đáp như sau chủ sở hữu sáng chế, kiểu dáng công nghiệp, thiết kế bố trí phải trả thù lao cho các đồng tác giả khi không có thỏa thuận như thế nào? Câu hỏi của anh A.L.Q đến từ TP.HCM.'

""" 

spark.sql(query).write.parquet(data_write,
                        mode = "overwrite")

24/03/22 07:29:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [174]:
result_df = spark.read.parquet("gs://thuvienphapluat_crawl/2024/03/22/thu_vien_phap_luat")

In [175]:
result_df.printSchema()

root
 |-- Lawname: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Question: string (nullable = true)
 |-- Answer: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [176]:
result_df.show()

+--------+--------------------+--------------------+--------------------+
| Lawname|               Title|            Question|              Answer|
+--------+--------------------+--------------------+--------------------+
|phapluat|Chủ sở hữu sáng c...|Tôi có thắc mắc m...|[Các yếu tố hạn c...|
+--------+--------------------+--------------------+--------------------+



# RDD

In [177]:
df.select( 
    F.col('Lawname'),
    F.col('Title')
).limit(10).show()

+--------+--------------------+
| Lawname|               Title|
+--------+--------------------+
|phapluat|Bằng độc quyền gi...|
|phapluat|Quyền công bố tác...|
|phapluat|Thời hạn bảo hộ q...|
|phapluat|Giấy chứng nhận đ...|
|phapluat|Công nghệ mới thu...|
|phapluat|Chủ sở hữu bí mật...|
|phapluat|Bí mật về nhân th...|
|phapluat|Chỉ dẫn địa lý đồ...|
|phapluat|Biên bản cung cấp...|
|phapluat|Trong các hình th...|
+--------+--------------------+



In [178]:
rdd_course_body = df.select("Lawname","Title").rdd

In [179]:
rdd_course_body

MapPartitionsRDD[521] at javaToPython at NativeMethodAccessorImpl.java:0

In [180]:
rdd_course_body.take(1)

[Row(Lawname='phapluat', Title='Bằng độc quyền giải pháp hữu ích có hiệu lực bao lâu và bắt đầu có hiệu lực từ thời điểm nào theo quy định?')]

In [181]:
all_data = rdd_course_body.collect()

In [182]:
rdd_course_body.getNumPartitions()

1

In [183]:
rdd_course_body.repartition(4).getNumPartitions()

4

In [184]:
rdd_course_body.getNumPartitions()

1

### Map

In [185]:
def count_len(row):
    Lawname = row.Lawname
    Title = row.Title
    return (Lawname, Title)
        
rdd_course_body.map(count_len).take(10)

[('phapluat',
  'Bằng độc quyền giải pháp hữu ích có hiệu lực bao lâu và bắt đầu có hiệu lực từ thời điểm nào theo quy định?'),
 ('phapluat',
  'Quyền công bố tác phẩm là quyền nhân thân hay quyền tài sản? Tác giả có được chuyển nhượng quyền công bố tác phẩm không?'),
 ('phapluat',
  'Thời hạn bảo hộ quyền công bố tác phẩm văn học của tác giả là suốt cuộc đời tác giả đúng không?'),
 ('phapluat',
  'Giấy chứng nhận đăng ký nhãn hiệu có hiệu lực từ thời điểm nào? Hiệu lực của loại GCN này sau khi gia hạn được quy định ra sao?'),
 ('phapluat',
  'Công nghệ mới thuộc đối tượng được khuyến khích chuyển giao từ nước ngoài vào Việt Nam khi có đủ yếu tố nào?'),
 ('phapluat',
  'Chủ sở hữu bí mật kinh doanh không có quyền cấm người khác thực hiện các hành vi nào theo quy định?'),
 ('phapluat',
  'Bí mật về nhân thân là gì? Bí mật về nhân thân không thể bảo hộ với danh nghĩa bí mật kinh doanh đúng không?'),
 ('phapluat',
  'Chỉ dẫn địa lý đồng âm là gì? Giấy chứng nhận đăng ký chỉ dẫn địa lý đồn

In [187]:
def count_word(row):
    Lawname = row.Lawname
    for word in Lawname.split(" "):
        yield (word,1)
        
rdd_course_body.flatMap(count_word).take(10)

[('phapluat', 1),
 ('phapluat', 1),
 ('phapluat', 1),
 ('phapluat', 1),
 ('phapluat', 1),
 ('phapluat', 1),
 ('phapluat', 1),
 ('phapluat', 1),
 ('phapluat', 1),
 ('phapluat', 1)]

In [189]:
def filter_product(row):
    Title = row.Title
    return Title != "Chủ sở hữu sáng chế, kiểu dáng công nghiệp, thiết kế bố trí phải trả thù lao cho các đồng tác giả khi không có thỏa thuận như thế nào?"
        
( 
    rdd_course_body
        .filter(filter_product)
        .map(count_len)
        .take(10)
)

[('phapluat',
  'Bằng độc quyền giải pháp hữu ích có hiệu lực bao lâu và bắt đầu có hiệu lực từ thời điểm nào theo quy định?'),
 ('phapluat',
  'Quyền công bố tác phẩm là quyền nhân thân hay quyền tài sản? Tác giả có được chuyển nhượng quyền công bố tác phẩm không?'),
 ('phapluat',
  'Thời hạn bảo hộ quyền công bố tác phẩm văn học của tác giả là suốt cuộc đời tác giả đúng không?'),
 ('phapluat',
  'Giấy chứng nhận đăng ký nhãn hiệu có hiệu lực từ thời điểm nào? Hiệu lực của loại GCN này sau khi gia hạn được quy định ra sao?'),
 ('phapluat',
  'Công nghệ mới thuộc đối tượng được khuyến khích chuyển giao từ nước ngoài vào Việt Nam khi có đủ yếu tố nào?'),
 ('phapluat',
  'Chủ sở hữu bí mật kinh doanh không có quyền cấm người khác thực hiện các hành vi nào theo quy định?'),
 ('phapluat',
  'Bí mật về nhân thân là gì? Bí mật về nhân thân không thể bảo hộ với danh nghĩa bí mật kinh doanh đúng không?'),
 ('phapluat',
  'Chỉ dẫn địa lý đồng âm là gì? Giấy chứng nhận đăng ký chỉ dẫn địa lý đồn

In [190]:
def sort_by_value(record):
    return record[1]

(
    rdd_course_body
        .filter(filter_product)
        .map(count_len)
        .sortBy(sort_by_value,ascending = False)
        .take(10)
)


[('phapluat',
  'Tài liệu chứng minh quyền sở hữu nhãn hiệu nổi tiếng và chứng minh sự nổi tiếng của nhãn hiệu bao gồm những gì?'),
 ('phapluat',
  'Trong các hình thức chuyển giao công nghệ thì phần chuyển giao công nghệ có bao nhiêu trường hợp? Những trường hợp đó là gì?'),
 ('phapluat',
  'Thời hạn của bằng bảo hộ giống cây trồng phụ thuộc vào yếu tố gì? Mẫu tờ khai yêu cầu sửa đổi bằng bảo hộ giống cây trồng?'),
 ('phapluat',
  'Thời hạn bảo hộ quyền công bố tác phẩm văn học của tác giả là suốt cuộc đời tác giả đúng không?'),
 ('phapluat',
  'Ranh giới của khu vực địa lý mang chỉ dẫn địa lý được xác định thế nào? GCN đăng ký chỉ dẫn địa lý ghi nhận thông tin gì?'),
 ('phapluat',
  'Quyền công bố tác phẩm là quyền nhân thân hay quyền tài sản? Tác giả có được chuyển nhượng quyền công bố tác phẩm không?'),
 ('phapluat',
  'Phải nộp phí duy trì hiệu lực Bằng độc quyền giải pháp hữu ích khi đề nghị cấp bằng hay sau khi bằng bảo hộ hết hạn hiệu lực?'),
 ('phapluat',
  'Nhãn hiệu chứng nh

### Reduce

In [191]:
def my_sum(x,y):
    return x + y

In [192]:
(
    rdd_course_body
        .map(count_len)
        .reduceByKey(my_sum)
        .sortBy(sort_by_value,ascending = False)
        .take(10)
)

[('phapluat',
  'Bằng độc quyền giải pháp hữu ích có hiệu lực bao lâu và bắt đầu có hiệu lực từ thời điểm nào theo quy định?Quyền công bố tác phẩm là quyền nhân thân hay quyền tài sản? Tác giả có được chuyển nhượng quyền công bố tác phẩm không?Thời hạn bảo hộ quyền công bố tác phẩm văn học của tác giả là suốt cuộc đời tác giả đúng không?Giấy chứng nhận đăng ký nhãn hiệu có hiệu lực từ thời điểm nào? Hiệu lực của loại GCN này sau khi gia hạn được quy định ra sao?Công nghệ mới thuộc đối tượng được khuyến khích chuyển giao từ nước ngoài vào Việt Nam khi có đủ yếu tố nào?Chủ sở hữu bí mật kinh doanh không có quyền cấm người khác thực hiện các hành vi nào theo quy định?Bí mật về nhân thân là gì? Bí mật về nhân thân không thể bảo hộ với danh nghĩa bí mật kinh doanh đúng không?Chỉ dẫn địa lý đồng âm là gì? Giấy chứng nhận đăng ký chỉ dẫn địa lý đồng âm có có hiệu lực vô thời hạn đúng không?Biên bản cung cấp mẫu giám định tư pháp về quyền tác giả, quyền liên quan được thực hiện theo mẫu nào?Tr

# World Count  bằng rdd

In [193]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

def count_word(row):
    name_count = row.Lawname
    for word in name_count.split(" "):
        yield (word,1)

def my_sum(x,y):
    return x + y

spark = ( 
    SparkSession.builder
            .master("yarn") 
            .appName('Word Count')
            .getOrCreate()
)

schema = types.StructType([
    types.StructField('Link', types.StringType(), True),
    types.StructField('Lawname', types.StringType(), True),
    types.StructField('Title', types.StringType(), True),
    types.StructField('Question', types.StringType(), True),
    types.StructField('Answer', types.StringType(), True)

])

data_storage = "gs://thuvienphapluat_crawl/2024/03/22/HuynhHuuQuoc_First.parquet"
data_write = "gs://thuvienphapluat_crawl/2024/03/22/word_count"

df = spark.read.schema(schema).parquet(data_storage)
rdd_course_body = df.select("Lawname").rdd
(
    rdd_course_body
        .flatMap(count_word)
        .reduceByKey(my_sum)
        .saveAsTextFile(data_write)
)


24/03/22 07:40:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [194]:
from google.cloud import storage

def download_blob(bucket_name, source_blob_name):
    """Downloads a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    print(blob.download_as_text())

bucket_name = "thuvienphapluat_crawl"
source_blob_name = "2024/03/22/word_count/part-00000"
download_blob(bucket_name, source_blob_name)


('phapluat', 19)

